In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec

import uts

Read Train Data

In [2]:
df = pd.read_csv('data/Form relationships.csv')
gdf = uts.process_data(df)
gdf.head()

/home/sunny/miniconda3/envs/env5/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,COLUMN_NAME,TABLE_NAME,ORDINAL_POSITION
0,Data Protection Key Id,__CDDataProtectionKeys,1
1,Friendly Name,__CDDataProtectionKeys,2
2,Xml,__CDDataProtectionKeys,3
3,System Name,__CDDataProtectionKeys,4
4,Migration Id,__MigrationHistory,1
...,...,...,...
2567,User Profile Event Id,UserProfileEvent,1
2568,Event Type,UserProfileEvent,2
2569,Event Date Time,UserProfileEvent,3
2570,Event Payload,UserProfileEvent,4


Read Test Data

In [3]:
ddf = pd.read_csv('data/Form relationships_dummy tables.csv')
gdf_test = uts.process_data(ddf)
gdf_test.get_group('A')

/home/sunny/miniconda3/envs/env5/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,COLUMN_NAME,TABLE_NAME,ORDINAL_POSITION
0,Ontology Audit Id,A,1
1,Ontology Event Id,A,2
2,Action,A,3
3,Action Date,A,4
4,Action User Profile Id,A,5
5,Field,A,6
6,Field Name,A,7
7,Previous Value,A,8
8,New Value,A,9
9,Ontology Id,A,10


Processing Train Dataset

In [4]:
# option_tb 2 : keep table name multiple times
# option_tb 1 : keep table name once
# option_tb 0 : drop table name

# option_p 1 : keep ordinal position
# option_p 0 : drop ordinal position

train_data = uts.df_to_sentence(gdf,1,0)
train_data[0]

{'AgentAction': 'Agent Action Id Micro Service Agent Id Name Description ML Data Type Id Micro Service Implementation AgentAction'}

In [5]:
tagged_data_train = uts.tagging_data(train_data)
tagged_data_train[0]

TaggedDocument(words=['Agent', 'Action', 'Id', 'Micro', 'Service', 'Agent', 'Id', 'Name', 'Description', 'ML', 'Data', 'Type', 'Id', 'Micro', 'Service', 'Implementation', 'AgentAction'], tags=['AgentAction'])

Processing Test Dataset

In [6]:
# iterate form -> sentence -> concatenat into a list [,,]
test_data = uts.df_to_sentence(gdf_test,1,0)
test_data[0]

{'A': 'Ontology Audit Id Ontology Event Id Action Action Date Action User Profile Id Field Field Name Previous Value New Value Ontology Id A'}

In [7]:
tagged_data_test= uts.tagging_data(test_data)
tagged_data_test[0]

TaggedDocument(words=['Ontology', 'Audit', 'Id', 'Ontology', 'Event', 'Id', 'Action', 'Action', 'Date', 'Action', 'User', 'Profile', 'Id', 'Field', 'Field', 'Name', 'Previous', 'Value', 'New', 'Value', 'Ontology', 'Id', 'A'], tags=['A'])

Combine Train + Test Data

In [8]:
comb_data = train_data+test_data
tagged_data_comb = uts.tagging_data(comb_data)

Train w/ Train data

In [9]:
#e: epochs, v: vector_size
#model = train(tagged_data_train, e = 500, v= 50) 
model = uts.train(tagged_data_comb, 500, 50) 

Result

In [10]:
# Pulling similar form ranking of form 'OntologyAudit'
model.docvecs.most_similar('OntologyAudit')


[('A', 0.9918113350868225),
 ('A4', 0.9873157143592834),
 ('A2', 0.9833207130432129),
 ('A1', 0.9806673526763916),
 ('A3', 0.9781286120414734),
 ('A+B', 0.8594748973846436),
 ('OntologyAttachment', 0.7464991807937622),
 ('OntologyEntityAudit', 0.726453959941864),
 ('B', 0.7218414545059204),
 ('B1', 0.7109195590019226)]

In [11]:
# Pulling similar form ranking of form 'UserProfile'
similar_doc = model.docvecs.most_similar('UserProfile')
similar_doc

[('UserPreference', 0.7321043014526367),
 ('Role', 0.6305222511291504),
 ('HTMLDocument', 0.6266763210296631),
 ('ScheduledTask', 0.5884966254234314),
 ('ComposeGroup', 0.5879925489425659),
 ('RoleEvent', 0.5769785046577454),
 ('Topic', 0.5682374835014343),
 ('ClassificationDomain', 0.567768931388855),
 ('RoleAudit', 0.557253360748291),
 ('Grammar', 0.5543255805969238)]

Load saved model

In [12]:
loaded_model = Doc2Vec.load("data/d2v.model")

Pairwise matrix for train data

In [13]:
tables = pd.DataFrame(gdf).loc[:,0]

In [14]:
print (loaded_model.docvecs.similarity('UserProfile', 'UserPreference'))
print (loaded_model.docvecs.distance('UserProfile', 'UserPreference'))

0.73210436
0.2678956389427185


Pairwise matrix for test data

In [15]:
test_tables = pd.DataFrame(gdf_test).loc[:,0]

In [16]:
# Making table name reference
dummy_table = {}

for i, val in enumerate(test_tables):
    dummy_table[val] = i

dummy_table['A']

0

Calculate dist A-A1 method 1

In [17]:
score_ = uts.cal_score_unseen('A2', 'A4', model, test_data, dummy_table)

Calculate dist A-A1 method 2 - result is the same

In [18]:
score_ = uts.cal_score_cos('A2', 'A4', model, test_data, dummy_table, alpha=0.1, min_alpha=0.0001,steps=5)

Matrix for Train data

In [19]:
# table x table score matrix
lst = []

for x in tables:
    score_x = []
    for y in tables:
        if x == y:
            score_x.append(0) # res dist is not exactly 0. Set to 0 here.
        else:
            score_x.append(loaded_model.docvecs.distance(x,y)) 
            
    lst.append(score_x)
    

In [20]:
# list to df
matrix = pd.DataFrame.from_records(lst)
matrix.set_axis(tables, axis=1, inplace=True)
matrix.rename(tables, inplace=True)
matrix.head()

,AgentAction,AgentActionAudit,AgentActionEvent,AgentActionImplementation,AgentActionImplementationAudit,AgentActionImplementationEvent,AgentObservable,AgentObservableAudit,AgentObservableEvent,AgentVersion,...,TreeVariableDeclaration,TreeVariableDeclarationAudit,TreeVariableDeclarationEvent,TreeVariableEvent,UserPreference,UserProfile,UserProfileAudit,UserProfileEvent,__CDDataProtectionKeys,__MigrationHistory
AgentAction,0.000000,0.715577,0.882634,0.347428,0.454003,0.587428,0.301082,0.506676,0.635506,0.479887,...,0.569545,0.685142,0.836026,1.045520,0.741065,0.909986,1.229810,1.274357,0.492537,0.457165
AgentActionAudit,0.715577,0.000000,0.212569,0.356461,0.208650,0.324099,0.657179,0.271253,0.532286,0.546231,...,0.928484,0.625587,0.832569,0.855700,0.788226,0.972069,0.608378,0.794957,0.939134,0.726135
AgentActionEvent,0.882634,0.212569,0.000000,0.549707,0.538943,0.178017,0.853217,0.566124,0.319763,0.736128,...,1.103551,0.934437,0.654446,0.619058,0.912505,1.083315,0.736515,0.517637,1.102345,0.889783
AgentActionImplementation,0.347428,0.356461,0.549707,0.000000,0.199525,0.324229,0.242547,0.305064,0.455158,0.326608,...,0.658894,0.654397,0.816389,1.045270,0.490589,0.712875,1.023573,1.078900,0.592937,0.363492
AgentActionImplementationAudit,0.454003,0.208650,0.538943,0.199525,0.000000,0.309518,0.395412,0.191490,0.546542,0.468943,...,0.757505,0.563131,0.894443,1.063692,0.613582,0.730451,0.716060,0.962926,0.784619,0.558047


In [21]:
# df to csv
matrix.to_csv('result/result_matrix.csv')

Matrix for Test data

In [22]:
# table x table score matrix
# distance = 1 - similarity score
lst_test = []

for x in test_tables:
    score_x = []
    for y in test_tables:
        if x == y:
            score_x.append(0) # res dist is not exactly 0. Set to 0 here.
        else:
            dist = 1 - (loaded_model.docvecs.similarity_unseen_docs(loaded_model, [x],[y])) #np.abs
            score_x.append(dist) 
            
    lst_test.append(score_x)

In [23]:
# list to df
matrix_test = pd.DataFrame.from_records(lst_test)
matrix_test.set_axis(test_tables, axis=1, inplace=True)
matrix_test.rename(test_tables, inplace=True)
matrix_test.head()

,A,A+B,A1,A2,A3,A4,B,B1,C,D
A,0.000000,1.033015,1.014507,1.188577,1.092923,1.033058,0.889648,1.013223,1.035516,0.870767
A+B,1.033015,0.000000,1.212400,0.785388,0.892105,1.228608,0.878109,0.838029,0.987368,1.095183
A1,1.014507,1.212400,0.000000,1.050145,1.034678,1.037129,1.104812,1.123838,0.998066,0.923063
A2,1.188577,0.785388,1.050145,0.000000,1.003367,1.320668,0.950800,0.780700,1.192678,1.116340
A3,1.092923,0.892105,1.034678,1.003367,0.000000,0.889323,0.992034,0.787728,0.919608,0.814573


Distance Tree for Training Data

In [24]:
from skbio import DistanceMatrix
from skbio.tree import nj
#run: pip install scikit-bio

In [25]:
dm = DistanceMatrix(matrix, tables)
tree = nj(dm)
str_tree = tree.ascii_art()

In [26]:
text_file = open("result/tree_train.txt", "w")
text_file.write(str_tree)
text_file.close()

Distance Tree for Test Data

In [27]:
dm = DistanceMatrix(matrix_test, test_tables)
tree = nj(dm)
str_tree = tree.ascii_art()

text_file = open("result/tree_test.txt", "w")
text_file.write(str_tree)
text_file.close()